In [ ]:
!git clone https://github.com/sithu31296/semantic-segmentation
%cd semantic-segmentation
%pip install -e .

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torchvision import io
from torchvision import transforms as T
from PIL import Image
import pickle
import matplotlib.pyplot as plt

In [16]:
from semseg import show_models

show_models()

  No.  Model Names
-----  -------------
    1  SegFormer
    2  Lawin
    3  SFNet
    4  BiSeNetv1
    5  DDRNet
    6  FCHarDNet
    7  BiSeNetv2


In [ ]:
%pip install -U gdown

In [ ]:
import gdown
from pathlib import Path

ckpt = Path('./checkpoints/pretrained/segformer')
ckpt.mkdir(exist_ok=True, parents=True)

url = 'https://drive.google.com/uc?id=1-OmW3xRD3WAbJTzktPC-VMOF5WMsN8XT'
output = './checkpoints/pretrained/segformer/segformer.b3.ade.pth'

gdown.download(url, output, quiet=False)

In [54]:
from semseg.models import *

model = eval('SegFormer')(
    backbone='MiT-B3',
    num_classes=150
)

try:
    model.load_state_dict(torch.load('checkpointcheckpoints/pretrained/segformer/segformer.b3.ade.pth', map_location='cpu'))
except:
    print("Download a pretrained model's weights from the result table.")
model.eval()

print('Loaded Model')

Download a pretrained model's weights from the result table.
Loaded Model


In [ ]:
model.eval()

In [ ]:
# Load the data loader from the pickle file

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class Ai4MarsData(Dataset):
    #X tensor (torch) -> images
    #y tensor (torch) -> labels

    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        image = self.X[index]
        label = self.y[index]

        if self.transform:
            image = self.transform(image)

        return image, label
    

with open('/content/drive/MyDrive/Dataset/data_loader.pkl', 'rb') as f:
    data_loader = pickle.load(f)


items = data_loader['dataloader'].dataset.__getitem__(1)

print(items[0].shape)


plt.imshow(items[0])
plt.show()

In [30]:
loader = data_loader['dataloader'].dataset

In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = model.to(device)

# Use torch.no_grad() to disable gradient computation during testing
with torch.no_grad():

    for images, labels in loader:
        # Move the data to the desired device
        images = images.permute(2,0,1).to(device)
        print(images.shape)
        #labels = labels.to(device)

        # Forward pass to get the predictions
        predictions = model(images)

torch.Size([3, 224, 224])


ValueError: ignored